In [1]:
from syllablecounter import load_model, get_data

In [2]:
def printmd(string, size = 3):
    from IPython.display import Markdown, display
    display(Markdown(f'<font size = {size}>{string}</font>'))
def green(string, deg = 1):
    return f'<span style = "background-color:hsl(125, {deg * 100}%, 50%);">{string}</span>'
def red(string, deg = 1):
    return f'<span style = "background-color:hsl(0, {deg * 100}%, 50%);">{string}</span>'
def ctext(string, deg = 0.5):
    deg = 2 * deg - 1
    if deg > 0:
        return green(string, deg)
    else:
        return red(string, -deg)

In [3]:
counter = load_model()
counter

SyllableCounter(
  (embed): Embedding(30, 64)
  (rnn): GRU(64, 128, num_layers=3, dropout=0.2, bidirectional=True)
  (drops): ModuleList(
    (0): Dropout(p=0.5, inplace=False)
    (1): Dropout(p=0.5, inplace=False)
  )
  (norms): ModuleList(
    (0): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (lins): ModuleList(
    (0): Linear(in_features=256, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=256, bias=True)
  )
  (out): Linear(in_features=256, out_features=1, bias=True)
)

In [4]:
import pandas as pd
df = pd.read_csv('data/gutsyls.tsv', sep = '\t')
df.head()

,words,syl_seq,syls
0,nondistillation,100100100101000,5
1,transitoriness,10001010101000,5
2,bipyramidal,10101001100,5
3,nonciteable,10010001100,4
4,livebearer,1000100010,3


In [5]:
hard_words = {'words': [], 'syl_seq': [], 'syls': []}
for idx in range(1000):
    yhat = counter.predict(df['words'][idx], return_confidence = True, return_sequence = True)
    if yhat['num_syls'] != df['syls'][idx]:
        hard_words['words'].append(df['words'][idx])
        hard_words['syl_seq'].append(df['syl_seq'][idx])
        hard_words['syls'].append(df['syls'][idx])
        print(df['words'][idx], df['syl_seq'][idx], df['syls'][idx], yhat['num_syls'], yhat['confidence'])
        print(yhat['probs'])

siobhan 1001000 2 3 0.4
[('s', 0.9), ('i', 0.12), ('o', 0.89), ('b', 0.87), ('h', 0.15), ('a', 0.12), ('n', 0.12)]
init 1000 1 2 0.56
[('i', 0.89), ('n', 0.19), ('i', 0.86), ('t', 0.11)]
bandaranaike 100101010110 6 4 0.22
[('b', 0.9), ('a', 0.11), ('n', 0.11), ('d', 0.9), ('a', 0.12), ('r', 0.17), ('a', 0.86), ('n', 0.89), ('a', 0.13), ('i', 0.11), ('k', 0.11), ('e', 0.12)]
lemonnier 101001000 3 4 0.28
[('l', 0.9), ('e', 0.12), ('m', 0.82), ('o', 0.22), ('n', 0.11), ('n', 0.9), ('i', 0.12), ('e', 0.89), ('r', 0.12)]
endostea 10100101 4 3 0.37
[('e', 0.9), ('n', 0.11), ('d', 0.9), ('o', 0.12), ('s', 0.13), ('t', 0.87), ('e', 0.13), ('a', 0.12)]
tandoori 10010001 3 4 0.29
[('t', 0.9), ('a', 0.11), ('n', 0.11), ('d', 0.89), ('o', 0.13), ('o', 0.68), ('r', 0.89), ('i', 0.14)]
penitente 100110010 4 3 0.33
[('p', 0.9), ('e', 0.11), ('n', 0.14), ('i', 0.88), ('t', 0.89), ('e', 0.14), ('n', 0.11), ('t', 0.1), ('e', 0.1)]
molise 101010 3 2 0.38
[('m', 0.9), ('o', 0.12), ('l', 0.25), ('i', 0.81)

In [ ]:
while True:
    words = input()
    outputs = counter.predict(words, return_sequence = True, pred_threshold = 0.5)
    string = ''.join([ctext(f'{char} ', deg) for char, deg in outputs['probs']])
    printmd(f'{string}&nbsp;= {outputs["num_syls"]} syllables', size = 5)